`Pair Programming ETL IV carga I`

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.

📌 Nota Todo lo tendremos que hacer desde jupyter notebook
Cread la BBDD con el nombre de tiburones.
Cread las tablas de la BBDD:
Tabla ataques
Tabla clima


BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación
🚨 En caso de que no tengáis los datos unidos de la sesión anterior, tenéis un csv datos_union_clima_ataques con todos los datos que necesitareis para este ejercicio

In [324]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
pd.options.display.max_columns=None

df= pd.read_csv('../Datos/etl_transformacion.csv')




In [325]:
df.head()

,year,type,country,area,location,activity,name,time,species,siglo,injury,mes,fatal,genero,categorias_tiburones,edades,wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m_direction,wind10m_speed
0,2018,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,18h00,White shark,siglo - XXI,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,291.250000,287.343750,285.468750,281.484375,279.609375,276.8750,274.218750,271.953125,270.234375,268.437500,266.093750,264.531250,263.046875,261.250000,259.37500,261.015625,9.359375,8.953125,8.171875,7.468750,7.015625,6.578125,6.234375,6.015625,5.812500,5.656250,5.531250,5.421875,5.281250,5.078125,4.968750,4.984375,11.500000,12.000000,10.109375,5.734375,3.968750,3.203125,2.687500,2.65625,3.312500,4.843750,5.937500,7.015625,7.718750,8.093750,7.500000,1.734375,97.5,7.515625,-9999.0,-9999.0,-9999.0,10.87500,7.015625,11.796875,1021.125,2.734375,0.0,296.796875,3.953125
1,2018,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,17h00,"Bull shark, 6'",siglo - XXI,Lower left leg bitten,May,N,M,Bull,15.0,291.250000,287.343750,285.468750,281.484375,279.609375,276.8750,274.218750,271.953125,270.234375,268.437500,266.093750,264.531250,263.046875,261.250000,259.37500,261.015625,9.359375,8.953125,8.171875,7.468750,7.015625,6.578125,6.234375,6.015625,5.812500,5.656250,5.531250,5.421875,5.281250,5.078125,4.968750,4.984375,11.500000,12.000000,10.109375,5.734375,3.968750,3.203125,2.687500,2.65625,3.312500,4.843750,5.937500,7.015625,7.718750,8.093750,7.500000,1.734375,97.5,7.515625,-9999.0,-9999.0,-9999.0,10.87500,7.015625,11.796875,1021.125,2.734375,0.0,296.796875,3.953125
2,2018,Provoked,australia,Queensland,Cairns Aquarium,Feeding sharks,male,Morning,Grey reef shark,siglo - XXI,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,155.234375,141.093750,150.390625,147.500000,142.500000,148.1250,124.843750,95.625000,98.515625,96.796875,101.640625,97.265625,93.203125,91.328125,84.68750,73.359375,5.578125,5.171875,4.781250,4.390625,4.156250,4.093750,3.656250,3.406250,3.359375,3.093750,2.765625,2.687500,2.640625,2.453125,2.375000,2.328125,12.500000,11.765625,10.140625,9.031250,8.531250,8.796875,9.578125,10.68750,9.734375,8.218750,8.171875,7.453125,6.187500,5.875000,7.765625,8.234375,97.5,6.953125,-9999.0,-9999.0,-9999.0,28.15625,-2.828125,9.890625,-9999.000,4.000000,0.0,170.781250,2.312500
3,2018,Unprovoked,australia,Western Australia,"Surfers Point, Prevelly",Surfing,Rob Bruce,09h30,3 m shark,siglo - XXI,"No injury, knocked off board by shark",Apr,N,M,Unknow,60.0,155.234375,141.093750,150.390625,147.500000,142.500000,148.1250,124.843750,95.625000,98.515625,96.796875,101.640625,97.265625,93.203125,91.328125,84.68750,73.359375,5.578125,5.171875,4.781250,4.390625,4.156250,4.093750,3.656250,3.406250,3.359375,3.093750,2.765625,2.687500,2.640625,2.453125,2.375000,2.328125,12.500000,11.765625,10.140625,9.031250,8.531250,8.796875,9.578125,10.68750,9.734375,8.218750,8.171875,7.453125,6.187500,5.875000,7.765625,8.234375,97.5,6.953125,-9999.0,-9999.0,-9999.0,28.15625,-2.828125,9.890625,-9999.000,4.000000,0.0,170.781250,2.312500
4,2018,Unprovoked,south africa,Western Cape Province,"Robberg Beach, Plettenberg Bay",Paddle-skiing,male,09h40,"White shar

# Creación de base de datos

In [327]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host='127.0.0.1',
      user="root",
      password='AlumnaAdalab' 
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [328]:
mydb = crear_bbdd('tiburones')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


# Creación de tablas

In [329]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [330]:
df.columns

Index(['year', 'type', 'country', 'area', 'location', 'activity', 'name',
       'time', 'species', 'siglo', 'injury', 'mes', 'fatal', 'genero',
       'categorias_tiburones', 'edades', 'wi_direction950mb',
       'wi_direction900mb', 'wi_direction850mb', 'wi_direction800mb',
       'wi_direction750mb', 'wi_direction700mb', 'wi_direction650mb',
       'wi_direction600mb', 'wi_direction550mb', 'wi_direction500mb',
       'wi_direction450mb', 'wi_direction400mb', 'wi_direction350mb',
       'wi_direction300mb', 'wi_direction250mb', 'wi_direction200mb',
       'wi_speed200mb', 'wi_speed250mb', 'wi_speed300mb', 'wi_speed350mb',
       'wi_speed400mb', 'wi_speed450mb', 'wi_speed500mb', 'wi_speed550mb',
       'wi_speed600mb', 'wi_speed650mb', 'wi_speed700mb', 'wi_speed750mb',
       'wi_speed800mb', 'wi_speed850mb', 'wi_speed900mb', 'wi_speed950mb',
       'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb',
       'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb

In [331]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355 entries, 0 to 1354
Data columns (total 77 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  1355 non-null   int64  
 1   type                  1355 non-null   object 
 2   country               1355 non-null   object 
 3   area                  1349 non-null   object 
 4   location              1345 non-null   object 
 5   activity              1347 non-null   object 
 6   name                  1355 non-null   object 
 7   time                  1355 non-null   object 
 8   species               1301 non-null   object 
 9   siglo                 1355 non-null   object 
 10  injury                1354 non-null   object 
 11  mes                   1355 non-null   object 
 12  fatal                 1355 non-null   object 
 13  genero                1355 non-null   object 
 14  categorias_tiburones  1355 non-null   object 
 15  edades               

In [332]:
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `idataque` INT NOT NULL AUTO_INCREMENT,
  `year` INT NOT NULL,
  `type` VARCHAR (200) NOT NULL,
  `country` VARCHAR (200) NOT NULL, 
  `area` VARCHAR (200) NOT NULL,   
  `location` VARCHAR(200) NOT NULL, 
  `activity` VARCHAR (200) NOT NULL,
  `name` VARCHAR (200) NOT NULL, 
  `time` VARCHAR (200) NOT NULL, 
  `species` VARCHAR (200) NOT NULL,
  `siglo` VARCHAR (200) NOT NULL, 
  `injury` VARCHAR (200) NOT NULL, 
  `mes` VARCHAR (200) NOT NULL, 
  `fatal` VARCHAR (200) NOT NULL, 
  `genero` VARCHAR (200) NOT NULL, 
  `categorias_tiburones` VARCHAR (200) NOT NULL, 
  `edades` DECIMAL NOT NULL,
   PRIMARY KEY (`idataque`)) 
 
ENGINE = InnoDB;
'''


In [333]:
tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `idclima`INT NOT NULL AUTO_INCREMENT,
  `wi_direction950mb`DECIMAL NOT NULL,
  `wi_direction900mb`DECIMAL NOT NULL,
  `wi_direction850mb`DECIMAL NOT NULL, 
  `wi_direction800mb`DECIMAL NOT NULL,
  `wi_direction750mb`DECIMAL NOT NULL,
  `wi_direction700mb`DECIMAL NOT NULL, 
  `wi_direction650mb`DECIMAL NOT NULL,
  `wi_direction600mb`DECIMAL NOT NULL, 
  `wi_direction550mb`DECIMAL NOT NULL, 
  `wi_direction500mb`DECIMAL NOT NULL,
  `wi_direction450mb`DECIMAL NOT NULL, 
  `wi_direction400mb`DECIMAL NOT NULL, 
  `wi_direction350mb`DECIMAL NOT NULL,
  `wi_direction300mb`DECIMAL NOT NULL, 
  `wi_direction250mb`DECIMAL NOT NULL, 
  `wi_direction200mb`DECIMAL NOT NULL,
  `wi_speed200mb`DECIMAL NOT NULL,
  `wi_speed250mb`DECIMAL NOT NULL, 
  `wi_speed300mb`DECIMAL NOT NULL, 
  `wi_speed350mb`DECIMAL NOT NULL,
  `wi_speed400mb`DECIMAL NOT NULL, 
  `wi_speed450mb`DECIMAL NOT NULL, 
  `wi_speed500mb`DECIMAL NOT NULL, 
  `wi_speed550mb`DECIMAL NOT NULL,
  `wi_speed600mb`DECIMAL NOT NULL, 
  `wi_speed650mb`DECIMAL NOT NULL, 
  `wi_speed700mb`DECIMAL NOT NULL,
  `wi_speed750mb`DECIMAL NOT NULL,
  `wi_speed800mb`DECIMAL NOT NULL, 
  `wi_speed850mb`DECIMAL NOT NULL,
  `wi_speed900mb`DECIMAL NOT NULL, 
  `wi_speed950mb`DECIMAL NOT NULL,
  `rh_950mb`DECIMAL NOT NULL, 
  `rh_900mb`DECIMAL NOT NULL, 
  `rh_850mb`DECIMAL NOT NULL, 
  `rh_800mb`DECIMAL NOT NULL, 
  `rh_750mb`DECIMAL NOT NULL, 
  `rh_700mb`DECIMAL NOT NULL,
  `rh_650mb`DECIMAL NOT NULL, 
  `rh_600mb`DECIMAL NOT NULL, 
  `rh_550mb`DECIMAL NOT NULL, 
  `rh_500mb`DECIMAL NOT NULL, 
  `rh_450mb`DECIMAL NOT NULL, 
  `rh_400mb`DECIMAL NOT NULL,
  `rh_350mb`DECIMAL NOT NULL, 
  `rh_300mb`DECIMAL NOT NULL, 
  `rh_250mb`DECIMAL NOT NULL, 
  `rh_200mb`DECIMAL NOT NULL, 
  `timepoint`DECIMAL NOT NULL,
  `cloudcover`DECIMAL NOT NULL,
  `highcloud`DECIMAL NOT NULL, 
  `midcloud`DECIMAL NOT NULL, 
  `lowcloud`DECIMAL NOT NULL, 
  `temp2m`DECIMAL NOT NULL,
  `lifted_index`DECIMAL NOT NULL, 
  `rh2m`DECIMAL NOT NULL, 
  `msl_pressure`DECIMAL NOT NULL, 
  `prec_amount`DECIMAL NOT NULL, 
  `snow_depth`DECIMAL NOT NULL,
  `wind10m_direction` DECIMAL, 
  `wind10m_speed` DECIMAL,
  PRIMARY KEY (`idclima`))
ENGINE = InnoDB;
'''

In [334]:
crear_insertar_tabla("clima",'AlumnaAdalab', tabla_ataques)

In [335]:
crear_insertar_tabla("clima",'AlumnaAdalab', tabla_clima)

# Insercion de datos

In [336]:
# c_ataques = df.columns.values[:16]
# print(c_ataques)

In [337]:
# str_lista_columnas=""
# for i in c_ataques:
#     str_lista_columnas+="\"{fila[\'"+i+"\']}\","
# print(str_lista_columnas)


In [338]:
for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_ataques = f"""
                INSERT INTO ataques (year,type,country,area,location,activity,name,time,species,siglo,injury,mes,fatal,genero,categorias_tiburones,edades) 
                VALUES ( "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['area']}", "{fila['location']}", "{fila['activity']}", "{fila['name']}", "{fila['time']}", "{fila['species']}", "{fila['siglo']}", "{fila['injury']}", "{fila['mes']}", "{fila['fatal']}", "{fila['genero']}", "{fila['categorias_tiburones']}","{fila['edades']}");
                """

    crear_insertar_tabla("tiburones", 'AlumnaAdalab', query_ataques)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '8"", "siglo - XXI", "Left arm & right hand severed, lacerations to both legs", "' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '8"", "siglo - XXI", "Left arm & right hand severed, lacerations to both legs", "' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left hip", "Oct", "N", "M", "White","45.0")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left hip", "Oct", "N", "M", "White","45.0")' at line 2
1064 (42000): You have an erro

In [343]:
columns = ','.join(df.columns.values[16:])
print(columns)

wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m_direction,wind10m_speed


In [345]:
columns = df.columns.values[16:]

In [ ]:
str_lista_columnas=""
for i in columns:
    str_lista_columnas+="\"{fila[\'"+i+"\']}\","
print(str_lista_columnas)


In [347]:
for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_clima = f"""
                INSERT INTO clima (wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m_direction,wind10m_speed) 
                VALUES ( "{fila['wi_direction950mb']}","{fila['wi_direction900mb']}","{fila['wi_direction850mb']}","{fila['wi_direction800mb']}","{fila['wi_direction750mb']}","{fila['wi_direction700mb']}","{fila['wi_direction650mb']}","{fila['wi_direction600mb']}","{fila['wi_direction550mb']}","{fila['wi_direction500mb']}","{fila['wi_direction450mb']}","{fila['wi_direction400mb']}","{fila['wi_direction350mb']}","{fila['wi_direction300mb']}","{fila['wi_direction250mb']}","{fila['wi_direction200mb']}","{fila['wi_speed200mb']}","{fila['wi_speed250mb']}","{fila['wi_speed300mb']}","{fila['wi_speed350mb']}","{fila['wi_speed400mb']}","{fila['wi_speed450mb']}","{fila['wi_speed500mb']}","{fila['wi_speed550mb']}","{fila['wi_speed600mb']}","{fila['wi_speed650mb']}","{fila['wi_speed700mb']}","{fila['wi_speed750mb']}","{fila['wi_speed800mb']}","{fila['wi_speed850mb']}","{fila['wi_speed900mb']}","{fila['wi_speed950mb']}","{fila['rh_950mb']}","{fila['rh_900mb']}","{fila['rh_850mb']}","{fila['rh_800mb']}","{fila['rh_750mb']}","{fila['rh_700mb']}","{fila['rh_650mb']}","{fila['rh_600mb']}","{fila['rh_550mb']}","{fila['rh_500mb']}","{fila['rh_450mb']}","{fila['rh_400mb']}","{fila['rh_350mb']}","{fila['rh_300mb']}","{fila['rh_250mb']}","{fila['rh_200mb']}","{fila['timepoint']}","{fila['cloudcover']}","{fila['highcloud']}","{fila['midcloud']}","{fila['lowcloud']}","{fila['temp2m']}","{fila['lifted_index']}","{fila['rh2m']}","{fila['msl_pressure']}","{fila['prec_amount']}","{fila['snow_depth']}","{fila['wind10m_direction']}","{fila['wind10m_speed']}");
                """

    crear_insertar_tabla("tiburones", 'AlumnaAdalab', query_clima)